<a href="https://colab.research.google.com/github/LC1332/smooth-face-extract/blob/main/notebook/%E6%B5%8B%E8%AF%95FaceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 导入视频
- [ ] 选一个时间
- [ ] 看看detection能不能跑

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/SD/exp0226

expression_example.mp4


In [3]:
input_video = "/content/drive/MyDrive/SD/exp0226/expression_example.mp4"
target_folder = "/content/"

import shutil

shutil.copy(input_video, target_folder)

'/content/expression_example.mp4'

In [4]:
%cd /content
!rm -rf /content/smooth-face-extract
!git clone https://github.com/LC1332/smooth-face-extract


/content
Cloning into 'smooth-face-extract'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 48 (delta 23), reused 3 (delta 1), pack-reused 0
Receiving objects: 100% (48/48), 23.89 KiB | 3.98 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [5]:
%cd /content/smooth-face-extract/Face_detection_src
!ls
!mkdir face_images

/content/smooth-face-extract/Face_detection_src
combine_img_to_video.py        FaceDetection_MAFilter.py  __init__
FaceDetection_KalmanFilter.py  FaceDetection.py


In [6]:
!pip install -q mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 16.7 MB/s eta 0:00:00


In [7]:
!rm -rf face_images

!rm -rf output

In [8]:

!python FaceDetection_KalmanFilter.py --input_video_name /content/expression_example.mp4 \
    --start_time 00:43:55 \
    --end_time 00:44:04 \
    --maxcenter_speed 30


2024-02-29 01:18:30.573324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 01:18:30.573445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 01:18:30.576160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 01:18:30.593755: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 01:18:33.181182: W tensorflow/compiler/tf2

读取视频的一个frame

获得fps

获得start和end time


In [10]:
def get_frame_no(time_str, fps):
    h, m, s = map(int, time_str.split(":"))
    return int((h * 3600 + m * 60 + s) * fps)

In [9]:
input_video_name = "/content/expression_example.mp4"
start_time = "00:43:55"
end_time = "00:44:04"

import cv2

cap = cv2.VideoCapture(input_video_name)
fps = cap.get(cv2.CAP_PROP_FPS)

In [11]:
start_frame_no = get_frame_no(start_time, fps)
end_frame_no = get_frame_no(end_time, fps)

In [14]:
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection

face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)



In [20]:
print(dir(face_detection))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_packet_content', '_graph', '_graph_outputs', '_initialize_graph_interface', '_input_side_packets', '_input_stream_type_info', '_make_packet', '_modify_calculator_options', '_output_stream_type_info', '_set_extension', '_side_input_type_info', '_simulated_timestamp', 'close', 'create_graph_options', 'process', 'reset']


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_packet_content', '_graph', '_graph_outputs', '_initialize_graph_interface', '_input_side_packets', '_input_stream_type_info', '_make_packet', '_modify_calculator_options', '_output_stream_type_info', '_set_extension', '_side_input_type_info', '_simulated_timestamp', 'close', 'create_graph_options', 'process', 'reset']

In [33]:
!wget -q -O detector.tflite -q https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite

In [34]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a face detector instance with the video mode:
options = FaceDetectorOptions(
    base_options= BaseOptions( model_asset_path='detector.tflite'),
    running_mode=VisionRunningMode.VIDEO,
    min_detection_confidence=0.5)

face_detection = FaceDetector.create_from_options(options)


In [27]:
nt(options)

FaceDetectorOptions(base_options=BaseOptions(model_asset_path='/content/model.task', model_asset_buffer=None, delegate=None), running_mode=<VisionTaskRunningMode.VIDEO: 'VIDEO'>, min_detection_confidence=0.5, min_suppression_threshold=0.3, result_callback=None)
